# Route Guide System - Results Analysis

This notebook demonstrates how to analyze and visualize results from the Route Guide System.

## Setup

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from collections import Counter

# Matplotlib configuration
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 6)

print("Setup complete!")

## Load Route Data

Load a route guide result file from the results directory.

In [ ]:
# List available result files
results_dir = Path('../results')
result_files = list(results_dir.glob('route_*.json'))

print(f"Found {len(result_files)} result files:")
for i, f in enumerate(result_files[:5]):  # Show first 5
    print(f"{i+1}. {f.name}")

# Load the most recent file
if result_files:
    latest_file = max(result_files, key=lambda f: f.stat().st_mtime)
    print(f"\nLoading: {latest_file.name}")
    
    with open(latest_file, 'r') as f:
        route_data = json.load(f)
    
    print(f"\nRoute: {route_data['source']} → {route_data['destination']}")
    print(f"Stops: {len(route_data['stops'])}")
else:
    print("\nNo result files found. Run the route guide system first.")

## Basic Statistics

In [ ]:
if result_files:
    metadata = route_data['metadata']
    
    print("Route Statistics:")
    print(f"  Total Distance: {metadata.get('total_distance_km', 'N/A')} km")
    print(f"  Estimated Duration: {metadata.get('estimated_duration_minutes', 'N/A')} minutes")
    print(f"  Total Waypoints: {metadata.get('total_waypoints', 'N/A')}")
    print(f"  Processed Stops: {metadata.get('processed_stops', 'N/A')}")
    print(f"  Processing Time: {metadata.get('processing_time_seconds', 'N/A')} seconds")
    
    # Calculate per-stop processing time
    if metadata.get('processed_stops') and metadata.get('processing_time_seconds'):
        per_stop = metadata['processing_time_seconds'] / metadata['processed_stops']
        print(f"  Average Time per Stop: {per_stop:.2f} seconds")

## Content Type Distribution

Analyze which types of content (video, music, info) were selected most often.

In [ ]:
if result_files and route_data['stops']:
    # Count content types
    content_types = [stop['choice']['type'] for stop in route_data['stops']]
    type_counts = Counter(content_types)
    
    # Create DataFrame for visualization
    df_types = pd.DataFrame({
        'Type': list(type_counts.keys()),
        'Count': list(type_counts.values())
    })
    
    # Plot
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Bar chart
    colors = {'video': '#FF6B6B', 'music': '#4ECDC4', 'info': '#45B7D1'}
    bar_colors = [colors.get(t, '#95A5A6') for t in df_types['Type']]
    ax1.bar(df_types['Type'], df_types['Count'], color=bar_colors)
    ax1.set_xlabel('Content Type')
    ax1.set_ylabel('Number of Selections')
    ax1.set_title('Content Type Distribution')
    
    # Pie chart
    pie_colors = [colors.get(t, '#95A5A6') for t in df_types['Type']]
    ax2.pie(df_types['Count'], labels=df_types['Type'], autopct='%1.1f%%', 
            colors=pie_colors, startangle=90)
    ax2.set_title('Content Type Percentage')
    
    plt.tight_layout()
    plt.show()
    
    print("\nContent Type Summary:")
    for content_type, count in type_counts.items():
        percentage = (count / len(content_types)) * 100
        print(f"  {content_type.capitalize()}: {count} ({percentage:.1f}%)")

## Stop Details

View detailed information about each stop.

In [ ]:
if result_files and route_data['stops']:
    # Create DataFrame of stops
    stops_data = []
    for i, stop in enumerate(route_data['stops']):
        stops_data.append({
            'Stop #': i + 1,
            'Address': stop['address'][:50] + '...' if len(stop['address']) > 50 else stop['address'],
            'Type': stop['choice']['type'],
            'Title': stop['choice']['title'][:40] + '...' if len(stop['choice']['title']) > 40 else stop['choice']['title']
        })
    
    df_stops = pd.DataFrame(stops_data)
    display(df_stops)

## Selection Reasons Analysis

Analyze the reasons why content was selected.

In [ ]:
if result_files and route_data['stops']:
    print("Selection Reasons:\n")
    for i, stop in enumerate(route_data['stops'][:5]):  # Show first 5
        print(f"Stop {i+1}: {stop['address'][:60]}")
        print(f"  Selected: {stop['choice']['type'].upper()}")
        print(f"  Reason: {stop['choice']['reason']}")
        print()

## Performance Analysis

If you have multiple route results, compare their performance.

In [ ]:
if len(result_files) > 1:
    # Load all results for comparison
    all_results = []
    for file in result_files[:10]:  # Limit to 10 most recent
        with open(file, 'r') as f:
            data = json.load(f)
            all_results.append({
                'File': file.stem,
                'Stops': data['metadata'].get('processed_stops', 0),
                'Distance (km)': data['metadata'].get('total_distance_km', 0),
                'Processing Time (s)': data['metadata'].get('processing_time_seconds', 0)
            })
    
    df_results = pd.DataFrame(all_results)
    
    # Plot processing time vs number of stops
    plt.figure(figsize=(10, 6))
    plt.scatter(df_results['Stops'], df_results['Processing Time (s)'], 
                alpha=0.6, s=100, c='#3498DB')
    plt.xlabel('Number of Stops')
    plt.ylabel('Processing Time (seconds)')
    plt.title('Processing Time vs Number of Stops')
    plt.grid(True, alpha=0.3)
    plt.show()
    
    print("Performance Summary:")
    print(f"  Average Stops: {df_results['Stops'].mean():.1f}")
    print(f"  Average Processing Time: {df_results['Processing Time (s)'].mean():.1f}s")
    print(f"  Average Distance: {df_results['Distance (km)'].mean():.1f} km")

## Export Analysis

Export analysis results to CSV for further processing.

In [ ]:
if result_files and route_data['stops']:
    # Create detailed export
    export_data = []
    for stop in route_data['stops']:
        export_data.append({
            'Route': f"{route_data['source']} → {route_data['destination']}",
            'Address': stop['address'],
            'Content Type': stop['choice']['type'],
            'Title': stop['choice']['title'],
            'Content': stop['choice']['content'],
            'Reason': stop['choice']['reason']
        })
    
    df_export = pd.DataFrame(export_data)
    
    # Save to CSV
    export_file = results_dir / 'analysis_export.csv'
    df_export.to_csv(export_file, index=False)
    print(f"Analysis exported to: {export_file}")
    
    # Show preview
    display(df_export.head())

## Conclusions

This notebook provides basic analysis and visualization of Route Guide System results.

### Potential Extensions:
- Geographic visualization using folium or plotly
- Sentiment analysis of content titles and reasons
- Machine learning to predict best content type for locations
- Time series analysis of system performance
- A/B testing of different agent configurations